PREPROCESAMIENTO DE INFORMACIÓN DE LENDING CLUB

In [4]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
#Librerías
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('accepted_2007_to_2018Q4.csv/accepted_2007_to_2018Q4.csv', low_memory=False)

# Después de cargar el DataFrame, agrega esta línea para ver todas las columnas
print("Columnas en el DataFrame:")
print(df.columns.tolist())

Columnas en el DataFrame:
['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status

In [9]:

#Arreglo con las variables de interes
variables_solicitud = [
    'annual_inc',
    'avg_cur_bal',
    'desc',
    'emp_length',
    'emp_title',
    'home_ownership',
    'purpose',
    'installment',
    'loan_amnt',
    'title',
    'zip_code',
    'addr_state',
    'earliest_cr_line',
    'application_type',
    'verification_status',
    'open_acc',
    'pub_rec',
    'total_acc',
    'initial_list_status',
    'disbursement_method',
    'term',
    'int_rate'
]

# Verificar columnas existentes
columnas_faltantes = [col for col in variables_solicitud if col not in df.columns]
if columnas_faltantes:
    print("Las siguientes columnas no existen en el DataFrame:")
    print(columnas_faltantes)
else:
    df_solicitud = df[variables_solicitud]


Se procede a crear la transformación para cada variable.

Se propone que: 
- annual_inc, avg_cur_bal, installment, loan_amnt, int_rate se discreticen por rangos 
- home_ownership, purpose, application_type, verification_status, addr_state, initial_list_status, disbursement_method, term se codifique 
- emp_title, desc, title se vuelvan binarios
- open_acc, pub_rec, total_acc discretizar si tienen rango
- earliest_cr_line Transformarlo en "años desde apertura" (restar de la fecha actual), luego discretizar.
- zip_code para agruparlo por regiones

Nota: se explicará en el documento la razón por la cual se decidio hacer la transformación para las variables

In [17]:

# Paso 1: Cargar el archivo
df = pd.read_csv('accepted_2007_to_2018Q4.csv/accepted_2007_to_2018Q4.csv')

# Paso 2: Limpiar nombres de columnas
df.columns = df.columns.str.strip()
df = df.loc[:, ~df.columns.duplicated()]

# Paso 3: Filtrar columnas de interés
columnas_interes = [
    'annual_inc', 'avg_cur_bal', 'emp_length', 'emp_title', 'home_ownership',
    'purpose', 'installment', 'loan_amnt', 'title', 'zip_code',
    'application_type', 'verification_status', 'addr_state', 'open_acc',
    'pub_rec', 'total_acc', 'initial_list_status', 'disbursement_method',
    'earliest_cr_line', 'term', 'int_rate'
]

# Verificar qué columnas existen realmente
columnas_existentes = [col for col in columnas_interes if col in df.columns]
df_solicitud = df.loc[:, columnas_existentes]

# Paso 4: Variables de texto libre -> binarizar (si existen)
for col in ['emp_title', 'title']:
    if col in df_solicitud.columns:
        df_solicitud.loc[:, f'{col}_present'] = df_solicitud[col].notnull().astype(int)
        df_solicitud.drop(columns=col, inplace=True)

# Paso 5: Discretización de numéricas
def discretizar_variable(df, col, bins, labels):
    if col in df.columns:
        df.loc[:, f'{col}_bin'] = pd.cut(df[col], bins=bins, labels=labels, include_lowest=True)

discretizar_variable(df_solicitud, 'annual_inc', [0, 30000, 70000, 150000, np.inf], ['bajo', 'medio', 'alto', 'muy alto'])
discretizar_variable(df_solicitud, 'avg_cur_bal', [-1, 0, 5000, 20000, np.inf], ['sin saldo', 'bajo', 'medio', 'alto'])
discretizar_variable(df_solicitud, 'installment', [0, 250, 500, 1000, np.inf], ['bajo', 'medio', 'alto', 'muy alto'])
discretizar_variable(df_solicitud, 'loan_amnt', [0, 5000, 10000, 20000, np.inf], ['muy bajo', 'bajo', 'medio', 'alto'])
discretizar_variable(df_solicitud, 'int_rate', [0, 10, 15, 20, np.inf], ['baja', 'media', 'alta', 'muy alta'])

for col in ['open_acc', 'pub_rec', 'total_acc']:
    if col in df_solicitud.columns:
        df_solicitud.loc[:, f'{col}_bin'] = pd.cut(df_solicitud[col], [-1, 5, 10, 20, np.inf],
                                                   labels=['muy bajo', 'bajo', 'medio', 'alto'])

# Paso 6: Convertir earliest_cr_line a antigüedad (si existe)
# Convertir la columna a datetime (si existe)
if 'earliest_cr_line' in df_solicitud.columns:
    df_solicitud['earliest_cr_line'] = pd.to_datetime(df_solicitud['earliest_cr_line'], errors='coerce')
    
    # Verifica si la conversión fue exitosa (al menos un valor no nulo)
    if pd.api.types.is_datetime64_any_dtype(df_solicitud['earliest_cr_line']):
        df_solicitud['antiguedad_credito'] = datetime.now().year - df_solicitud['earliest_cr_line'].dt.year
        df_solicitud['antiguedad_credito_bin'] = pd.cut(df_solicitud['antiguedad_credito'],
                                                        [-1, 5, 10, 20, 40, np.inf],
                                                        labels=['<5', '5-10', '10-20', '20-40', '40+'])
        df_solicitud.drop(columns='earliest_cr_line', inplace=True)
    else:
        print("❌ Error: 'earliest_cr_line' no se pudo convertir a datetime.")


# Paso 7: Agrupar código postal (si existe)
if 'zip_code' in df_solicitud.columns:
    df_solicitud.loc[:, 'zip_prefix'] = df_solicitud['zip_code'].astype(str).str[:3]
    df_solicitud.drop(columns='zip_code', inplace=True)

# Paso 8: One-hot encoding
categorical_cols = [
    'home_ownership', 'purpose', 'application_type', 'verification_status',
    'addr_state', 'initial_list_status', 'disbursement_method', 'term',
    'annual_inc_bin', 'avg_cur_bal_bin', 'installment_bin', 'loan_amnt_bin',
    'int_rate_bin', 'open_acc_bin', 'pub_rec_bin', 'total_acc_bin',
    'antiguedad_credito_bin', 'zip_prefix'
]
# Elimina columnas que aparecen en menos del 1% de los datos
min_freq = len(df_solicitud) * 0.01
# Solo columnas numéricas para filtrar por frecuencia
numeric_cols = df_solicitud.select_dtypes(include=[np.number]).columns

# Calculamos la suma por columna numérica
column_sums = df_solicitud[numeric_cols].sum()

# Columnas que aparecen en al menos 1% de los casos (o que son todas cero)
cols_frecuentes = column_sums[(column_sums >= min_freq) | (column_sums == 0)].index

# Mantenemos esas columnas + todas las no numéricas
otras_cols = [col for col in df_solicitud.columns if col not in numeric_cols]
df_solicitud = df_solicitud[otras_cols + list(cols_frecuentes)]



# Solo usar columnas que realmente están
categorical_cols = [col for col in categorical_cols if col in df_solicitud.columns]

df_solicitud = pd.get_dummies(df_solicitud, columns=categorical_cols, drop_first=True)

# Paso 9: Mostrar el resultado
print(df_solicitud.head())


C:\Users\sarap\AppData\Local\Temp\ipykernel_44904\1196376637.py:2: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('accepted_2007_to_2018Q4.csv/accepted_2007_to_2018Q4.csv')
C:\Users\sarap\AppData\Local\Temp\ipykernel_44904\1196376637.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_solicitud['earliest_cr_line'] = pd.to_datetime(df_solicitud['earliest_cr_line'], errors='coerce')


  emp_length  annual_inc  avg_cur_bal  installment  loan_amnt  open_acc  \
0  10+ years     55000.0      20701.0       123.03     3600.0       7.0   
1  10+ years     65000.0       9733.0       820.28    24700.0      22.0   
2  10+ years     63000.0      31617.0       432.66    20000.0       6.0   
3  10+ years    110000.0      23192.0       829.90    35000.0      13.0   
4    3 years    104433.0      27644.0       289.91    10400.0      12.0   

   pub_rec  total_acc  int_rate  emp_title_present  ...  zip_prefix_991  \
0      0.0       13.0     13.99                  1  ...           False   
1      0.0       38.0     11.99                  1  ...           False   
2      0.0       18.0     10.78                  1  ...           False   
3      0.0       17.0     14.85                  1  ...           False   
4      0.0       35.0     22.45                  1  ...           False   

   zip_prefix_992  zip_prefix_993  zip_prefix_994  zip_prefix_995  \
0           False           F